In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from newrow import newrow_cash, newrow_etf_stock
from utils import broker_fee, get_date, round_half_up, get_asset_value
from fetch_data import fetch_exchange_rate
import requests
from secret import OXCR_KEY
from datetime import date, timedelta, datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cv = round_half_up(200)
print(cv)
print(41/cv * 4)

200.0
0.82


In [23]:
(dt - timedelta(1)).strftime("%Y-%m-%d")

'2025-10-10'

In [8]:
import yfinance as yf

# Crea un oggetto Ticker per il titolo Apple
apple = yf.Ticker("MWEQ.MI")

# Ottieni il codice ISIN
isin_code = apple.isin

print(f"Il codice ISIN di Apple (AAPL) è: {isin_code}")

Il codice ISIN di Apple (AAPL) è: -


## Importa il report:
- Per iniziare da zero con un report nuovo, importare `report-template.csv`
- Altrimenti importare `report.csv`
- Una volta svolte le operazioni, per salvare le modifiche seguire le istruzioni in fondo al notebook

In [5]:
from datetime import datetime, date

date_str = "01-10-2025"
date_obj = datetime.strptime(date_str, "%d-%m-%Y").strftime("%Y-%m-%d") # <-- .date() per ottenere un oggetto date

print(date.today())
print(date_obj)

print(date.today() == date_obj)  # Ora True se le date coincidono


2025-10-04
2025-10-01
False


In [4]:
print(8 + np.nan)

positions = []
print(sum(pos["value"] for pos in positions))


nan
0


In [49]:
df = pd.read_csv("reports/report copy.csv")  # Da zero: reports/report-template.csv

In [62]:
positions = get_asset_value(df, "AAPL", "2025-10-01")

print(positions)
print()
for item in positions:
    print(item)

/home/atonello/Code/Python/asset-management/utils.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  try:
[*********************100%***********************]  2 of 2 completed

[{'ticker': 'IONQ', 'quantity': 4.0, 'exchange_rate': 0.853078, 'price': np.float64(54.64331277122498), 'value': np.float64(218.5732510848999)}, {'ticker': 'QUBT', 'quantity': 8.0, 'exchange_rate': 0.853078, 'price': np.float64(16.404604939399718), 'value': np.float64(131.23683951519774)}]

{'ticker': 'IONQ', 'quantity': 4.0, 'exchange_rate': 0.853078, 'price': np.float64(54.64331277122498), 'value': np.float64(218.5732510848999)}
{'ticker': 'QUBT', 'quantity': 8.0, 'exchange_rate': 0.853078, 'price': np.float64(16.404604939399718), 'value': np.float64(131.23683951519774)}


In [63]:
nav = sum(pos["value"] for pos in positions)
print(nav)


349.8100906000976


In [25]:
import yfinance as yf

tickers = ["AAPL", "MSFT", "VOO"]
data = yf.download(tickers, period="1d")["Close"].iloc[-1]
data2 = yf.download(tickers, start="2025-10-01", end="2025-10-02")["Close"].iloc[-1]
print(data)
print(data2)

/tmp/ipykernel_113810/237958985.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period="1d")["Close"].iloc[-1]
[*********************100%***********************]  3 of 3 completed
/tmp/ipykernel_113810/237958985.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data2 = yf.download(tickers, start="2025-10-01", end="2025-10-02")["Close"].iloc[-1]
[*********************100%***********************]  3 of 3 completed

Ticker
AAPL    255.130005
MSFT    514.530029
VOO     612.549988
Name: 2025-10-01 00:00:00, dtype: float64
Ticker
AAPL    255.130005
MSFT    514.530029
VOO     612.549988
Name: 2025-10-01 00:00:00, dtype: float64


In [30]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def get_nav(positions, cash, ref_date):
    """
    Calcola il NAV del portafoglio al giorno di riferimento.
    
    positions: dict con ticker -> quantità
    cash: liquidità disponibile
    ref_date: string "YYYY-MM-DD"
    """
    tickers = list(positions.keys())
    date = datetime.strptime(ref_date, "%Y-%m-%d")
    
    # Proviamo a scaricare fino a 7 giorni indietro in caso di chiusura borsa
    for delta in range(0, 7):
        start = (date - timedelta(days=delta)).strftime("%Y-%m-%d")
        end = (date - timedelta(days=delta - 1)).strftime("%Y-%m-%d")
        data = yf.download(tickers, start=start, end=end, progress=False)
        if not data.empty:
            prices = data["Close"].iloc[-1]
            actual_date = start
            break
    else:
        raise ValueError("Nessun dato disponibile nei 7 giorni precedenti.")
    
    # Calcolo valori posizioni
    rows = []
    for ticker, qty in positions.items():
        price = prices[ticker]
        value = qty * price
        rows.append({"Ticker": ticker, "Quantità": qty, "Prezzo": price, "Valore": value})
    
    # Aggiungi cash
    rows.append({"Ticker": "CASH", "Quantità": "-", "Prezzo": "-", "Valore": cash})
    
    df = pd.DataFrame(rows)
    nav = df["Valore"].sum()
    
    return df, nav, actual_date

# Esempio d'uso
positions = {"AAPL": 10, "MSFT": 5, "VOO": 3}
cash = 2000.0
df, nav, actual_date = get_nav(positions, cash, "2023-12-31")  # 31 dicembre era domenica

print(f"Valutazione portafoglio al {actual_date}")
print(df)
print(f"\nNAV totale: {nav:.2f} USD")



/tmp/ipykernel_77214/1109819119.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start, end=end, progress=False)

3 Failed downloads:
['AAPL', 'VOO', 'MSFT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-31 -> 2024-01-01)')
/tmp/ipykernel_77214/1109819119.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start, end=end, progress=False)

3 Failed downloads:
['MSFT', 'VOO', 'AAPL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-30 -> 2023-12-31)')
/tmp/ipykernel_77214/1109819119.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start, end=end, progress=False)


Valutazione portafoglio al 2023-12-29
  Ticker Quantità      Prezzo       Valore
0   AAPL       10  190.913666  1909.136658
1   MSFT        5  371.209106  1856.045532
2    VOO        3  427.097351  1281.292053
3   CASH        -           -  2000.000000

NAV totale: 7046.47 USD


In [31]:
import yfinance as yf
import pandas as pd

tickers = ["AAPL", "MSFT", "VOO"]
ref_date = "2023-12-30"  # sabato, borsa chiusa

# Scarica un intervallo che include alcuni giorni prima
data = yf.download(tickers, start="2023-12-25", end="2023-12-31")["Close"]

# Prendi l'ultimo prezzo disponibile ≤ ref_date
last_prices = data.loc[data.index <= ref_date].iloc[-1]

print(last_prices)


/tmp/ipykernel_77214/4099571411.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2023-12-25", end="2023-12-31")["Close"]
[*********************100%***********************]  3 of 3 completed

Ticker
AAPL    190.913681
MSFT    371.209167
VOO     427.097260
Name: 2023-12-29 00:00:00, dtype: float64


## Visualizza il report:

In [75]:
df

,Data,Operazione,SIM,Prodotto,ISIN,Asset Name,TER,Valuta,QT. Scambio,Prezzo,Prezzo EUR,Imp. Nominale Operaz.,Commissioni,QT. Attuale Asset,PMPC Asset,Imp. Residuo Asset,Imp. Effettivo Operaz.,Costo Rilasciato,Plusv. Lorda,Minusv. Generata,Scadenza,Zainetto Fiscale,Plusv. Imponibile,Imposta,Netto,Liquidita Attuale,Valore Titoli,NAV,Liq. Storica Immessa
0,30-05-2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0
1,30-05-2025,deposit,BG Saxo,NaN,NaN,NaN,NaN,EUR,NaN,NaN,NaN,200.000000,NaN,NaN,NaN,NaN,200.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,200.000000,0.000000,200.000000,200.0
2,30-05-2025,buy,BG Saxo,Stock,US46222L1089,IONQ INC,NaN,USD,+4,-41.0000,-36.2829,-145.131472,0.882736,4.0,36.503552,146.014208,-146.014208,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,53.985792,145.131472,199.117264,200.0
3,10-06-2025,deposit,BG Saxo,NaN,NaN,NaN,NaN,EUR,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,153.985792,145.131472,299.117264,300.0
4,10-06-2025,buy,BG Saxo,Stock,US74766W1080,QUANTUM COMPUTING INC,NaN,USD,+9,-15.8779,-13.9365,-125.428154,0.875533,9.0,14.033743,126.303687,-126.303687,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,27.682105,270.559626,298.241731,300.0
5,16-06-2025,sell,BG Saxo,Stock,US74766W1080,QUANTUM COMPUTING INC,NaN,USD,-9,19.1320,16.4671,148.204106,0.858559,0.0,0.000000,0.000000,147.345547,126.303687,21.04186,0.0,NaN,0.0,21.04186,5.470884,15.570976,169.556768,144.255939,313.812707,300.0


In [74]:
fancy_df(df)

,Data,Operazione,SIM,Prodotto,ISIN,Asset Name,TER,Valuta,QT. Scambio,Prezzo,Prezzo EUR,Imp. Nominale Operaz.,Commissioni,QT. Attuale Asset,PMPC Asset,Imp. Residuo Asset,Imp. Effettivo Operaz.,Costo Rilasciato,Plusv. Lorda,Minusv. Generata,Scadenza,Zainetto Fiscale,Plusv. Imponibile,Imposta,Netto,Liquidita Attuale,Valore Titoli,NAV,Liq. Storica Immessa
0,30-05-2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,0.00,0.00,0.0
1,30-05-2025,deposit,BG Saxo,NaN,NaN,NaN,NaN,EUR,NaN,NaN,NaN,200.00,NaN,NaN,NaN,NaN,200.00,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,200.00,0.00,200.00,200.0
2,30-05-2025,buy,BG Saxo,Stock,US46222L1089,IONQ INC,NaN,USD,+4,-41.0000,-36.2829,-145.13,0.88,4.0,36.50,146.01,-146.01,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,53.99,145.13,199.12,200.0
3,10-06-2025,deposit,BG Saxo,NaN,NaN,NaN,NaN,EUR,NaN,NaN,NaN,100.00,NaN,NaN,NaN,NaN,100.00,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,153.99,145.13,299.12,300.0
4,10-06-2025,buy,BG Saxo,Stock,US74766W1080,QUANTUM COMPUTING INC,NaN,USD,+9,-15.8779,-13.9365,-125.43,0.88,9.0,14.03,126.30,-126.30,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,27.68,270.56,298.24,300.0
5,16-06-2025,sell,BG Saxo,Stock,US74766W1080,QUANTUM COMPUTING INC,NaN,USD,-9,19.1320,16.4671,148.20,0.86,0.0,0.00,0.00,147.35,126.3,21.04,0.0,NaN,0.0,21.04,5.47,15.57,169.56,144.26,313.81,300.0


### *Rimuovi manualmente l'ultima riga:*
- La cella qui sotto rimuove l'ultima riga del report. E' un "tasto elimina" manuale. 
- Ad esempio runnando la cella 3 volte rimuovo le ultime 3 righe

In [62]:
df = df.iloc[:-1]

## Programma principale
- Runnare la cella sottostante per avviare il programma e seguire le istruzioni a schermo

### IMPLEMENTATI:

1. ~~Liquidity~~
2. ~~ETF~~  check aggiuntivi
3. ~~Stock~~
4. `TODO` Bond
5. ~~Esci dal programma~~
- `todo` Controlli più strict sui broker (adesso se sbagli input defaulta a 0 fee)
- `TODO` Gestione separata intermediari 

In [66]:
def choice_etf_stock(df, choice="ETF"):
    
    date = get_date(df)
    brk = int(input("Seleziona intermediario (SIM) (1. Fineco, 2. BG Saxo): "))
    currency = int(input("Seleziona valuta (1. EUR, 2. USD): "))
    
    conv_rate = 1.0
    if currency == 2:
        conv_rate = float(input("Inserisci tasso di conversione: "))

    isin = input("Inserisci ISIN: ")

    qt = input("Inserisci quantità (intero positivo): ")
    if not (qt.isdigit() and int(qt) > 0):
        raise ValueError("Quantità non valida. Deve essere un intero positivo.")
    else:
        quantity = int(qt)

    price = float(input("Inserisci prezzo unitario (negativo se acquisto): "))
    if price == 0:
        raise ValueError("Il prezzo non può essere 0€.\nNegativo se acquisto, positivo se vendita.")
    price_og = price

    broker, fee = broker_fee(brk, choice, conv_rate, trade_value=quantity * abs(price_og*conv_rate))

    price = price * conv_rate

    if price < 0:   # buy
        buy = True
        difference = df["Liquidita Attuale"].iloc[-1] + (quantity * price - fee)
        #if difference < 0:
        #    raise ValueError(f"Liquidità insufficiente: {df["Liquidita Attuale"].iloc[-1]}€ {quantity * price - fee}€ = {difference}€")
    
    else:           # sell
        buy = False
    
    ter = np.nan
    if choice == "ETF":
        ter, err = fetch_ter(isin)
        if not ter:
            print(err)
            ter = input("Inserisci TER manualmente: ")

    return newrow_etf_stock(df, date, "EUR" if currency==1 else "USD", choice, isin, quantity, price_og, price, ter, broker, fee, buy)

In [71]:
def main_menu():
    print("\n1. Liquidità")
    print("2. ETF")
    print("3. Azioni")
    print("4. Obbligazioni")
    print("5. Visualizza statistiche...")
    print("6. Esci dal programma")

while True:
    main_menu()
    choice = input("Seleziona operazione (1 2 3 4 5 6): ")

    # CASH
    if choice == '1':

        date = get_date(df)

        brk = int(input("Seleziona intermediario (SIM) (1. Fineco, 2. BG Saxo): "))
        brokers = {
            1: "Fineco",
            2: "BG Saxo",
        }
        broker = brokers.get(brk, "SIM non riconosciuto")
        
        cash = float(input("Inserisci contante da depositare o prelevare: "))
        if cash == 0.0:
            raise ValueError("Il contante inserito non può essere 0€.\nPositivo se depositato, negativo se prelevato.")
        
        
        df = newrow_cash(df, date, cash, broker)
    
    # ETF
    elif choice == '2':
        df = choice_etf_stock(df, choice="ETF")

    
    elif choice == '3':
        df = choice_etf_stock(df, choice="Stock")

    elif choice == '4':
        print("Implementa Obbligazioni")

    elif choice == '5':
        pl = df["Netto"].sum()
        print(f"\nP&L Totale: {round_half_up(pl)}€")
        
    elif choice == '6':
        print("Esco dal programma...")
        break
    
    else:
        print("Scelta non valida. Riprova.")


1. Liquidità
2. ETF
3. Azioni
4. Obbligazioni
5. Visualizza statistiche...
6. Esci dal programma



1. Liquidità
2. ETF
3. Azioni
4. Obbligazioni
5. Visualizza statistiche...
6. Esci dal programma

1. Liquidità
2. ETF
3. Azioni
4. Obbligazioni
5. Visualizza statistiche...
6. Esci dal programma

1. Liquidità
2. ETF
3. Azioni
4. Obbligazioni
5. Visualizza statistiche...
6. Esci dal programma
Esco dal programma...


## Salva il report
- Di default sovrascrive `report.csv`
- Per salvare invece in nuovo file, cambiare `report.csv` nel nome desiderato

In [76]:
df.to_csv("reports/report.csv", index=False)    # Esempio nuovo file: reports/nuovo-report.csv
